In this notebook, I develop several visuals that are used in the lecture notes.

### 1. Gradient Clipping Visual

In [48]:
import numpy as np
import matplotlib.pyplot as plt

# Define the function to optimize
def f(x):
    return x**6 - x**2

def df(x):
    return 6*x**5 - 2*x

def gradient_descent(f, x0, alpha=0.1, n_iter=100, clip_norm=None):
    """Gradient descent algorithm."""
    x = x0
    x_history = []
    for i in range(n_iter):
        x_history.append(x)
        x -= alpha * df(x)
        if clip_norm is not None:
            norm = np.linalg.norm(x)
            if norm > clip_norm:
                x = x / norm * clip_norm
    return x_history

# Run gradient descent with and without clipping and plot the results
x0 = 1.2
x_history = gradient_descent(f, x0, alpha=0.5, n_iter=100)
x_history_clipped = gradient_descent(f, x0, alpha=0.5, n_iter=100, clip_norm=0.1)

# At each point of the trajectory, plot the gradient vector
x = np.linspace(0, 1.21, 100)
y = f(x)
plt.plot(x, y)
plt.plot(x_history, f(np.array(x_history)), 'o-')
plt.plot(x_history_clipped, f(np.array(x_history_clipped)), 'o-')
plt.legend(['f(x)', 'No clipping', 'Clipping'])
plt.xlabel('x')
plt.ylabel('y')
plt.show()


OverflowError: (34, 'Numerical result out of range')